<a href="https://colab.research.google.com/github/Ioana-P/IoanaFio/blob/main/content/project/twitter_sentiment_tracking/Using_Bertopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip install bertopic
# !pip install --upgrade joblib==1.1.0

In [1]:
from bertopic import *
import pandas as pd
from umap import UMAP

In [2]:
import sys
import resource

In [46]:


filepath = 'data/text_for_topics.csv'
docs = pd.read_csv(filepath, index_col=0)['clean_tweet_text'].to_list()


In [4]:
print(resource.getrlimit(resource.RLIMIT_STACK))

(8388608, -1)


In [3]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [1]:
%%timeit
#generating embeddings ahead of loading them into BertTopic

embeddings = sentence_model.encode(docs, show_progress_bar=True)

NameError: ignored

In [ ]:
import pickle

tweet_ids = pd.read_csv(filepath, index_col=0)['tweet_id'].to_list()

with open("data/sentence_embeddings.pkl", "wb") as fOut:
    pickle.dump({'tweet_id': tweet_ids, 'embeddings': embeddings}, fOut)

In [34]:
%%timeit

#now we can instantiate our BertTopic object and load in the umap model
#the embeddings and some params

RD_STATE = 12345
umap_model = UMAP(n_neighbors=20, n_components=5, 
                low_memory=True,
              min_dist=0.0, metric='cosine', random_state=RD_STATE)


topic_model = BERTopic(
    language="multilingual",
    umap_model = umap_model,
    min_topic_size = 10,
    nr_topics = 'auto',

    low_memory=True,
    #setting calc probs to True
    calculate_probabilities = True,
    )

In [35]:
%%timeit
# and now we finally fit and transform the data using the model
topics, probs = topic_model.fit_transform(docs, embeddings)

In [36]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,8634,-1_the_of_to_and
1,0,1017,0_牛b_omg_yessssssss_beauti
2,1,748,1_ukraine_russia_putin_russian
3,2,508,2_elon_elons_his_he
4,3,373,3_starship_spacex_falcon_launch
...,...,...,...
268,267,11,267_scale_master_18x6_20x4
269,268,10,268_bully_bullies_warmonger_sheepzero
270,269,10,269_shes_legs_her_girl
271,270,10,270_employees_qualifications_resignation_under...


In [42]:
viz_tops = topic_model.visualize_topics()


In [44]:
viz_tops.write_html("fig/viz_topics_22_10_13_1st.html")


In [41]:
topic_model.save('topic_model_22_10_13_1st')
topic_model.save()

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

